# WEB SCRAPING 

In [1]:
#Import the dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [2]:
#Create lists to store the scraped data
authors = []
dates = []
statements = []
sources = []
targets = []


# GETTING REAL NEWS FROM POLITIFACT WEBSITE


In [48]:
#Create a function to scrape the site
def scrape_website(page_number):
    page_num = str(page_number) #Convert the page number to a string
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num+'&ruling=true' #append the page number to complete the URL
    webpage = requests.get(URL)  #Make a request to the website
    time.sleep(3)
    soup = BeautifulSoup(webpage.text, "html.parser") #Parse the text from the website
    
    #Get the tags and it's class
    statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'})  #Get the tag and it's class
    statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) #Get the tag and it's class
    statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#Get the tag and it's class
    target = soup.find_all('div', attrs={'class':'m-statement__meter'}) #Get the tag and it's class
    
    #loop through the footer class m-statement__footer to get the date and author
    
    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name+' '+last_name
       # month = name_and_date[4]
      #  day = name_and_date[5]
       # year = name_and_date[6]
       # date = month+' '+day+' '+year
        #dates.append(date)
        authors.append(full_name)
        
    #Loop through the div m-statement__quote to get the link
    for i in statement_quote:
        link2 = i.find_all('a')  # only list can be converted to text
        statements.append(link2[0].text.strip())
    #Loop through the div m-statement__meta to get the source
    for i in statement_meta:
        link3 = i.find_all('a') #Source
        source_text = link3[0].text.strip()
        sources.append(source_text)
    #Loop through the target or the div m-statement__meter to get the facts about the statement (True or False)
    for i in target:
        fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(fact)

In [49]:
#Loop through 'n-1' webpages to scrape the data
n=100
for i in range(1, n):
    scrape_website(i)

In [50]:
#Create a new dataFrame 
data = pd.DataFrame(columns = ['author',  'statement', 'source', 'target']) 
data['author'] = authors
data['statement'] = statements
data['source'] = sources
#data['date'] = dates
data['target'] = targets
#Show the data set
data

,author,statement,source,target
0,Jill Terreri,"New York is ""leading the nation with the lowes...",NYS Department of Corrections and Community Su...,true
1,Eleanor Hildebrandt,“Iowa ranks 45th in the U.S. for internet conn...,Cindy Axne,true
2,Emily Zirkelbach,“FUN FACT: West Virginia is home to zero billi...,David Gura,true
3,Madeline Heim,Says gubernatorial candidate Rebecca Kleefisch...,Kelda Helen Roys,true
4,Sabine Martin,"The Biden administration ""published a study co...",Randy Feenstra,true
...,...,...,...,...
2430,Bill Adair,"""Corporate profits are up, CEO pay is up, but ...",Hillary Clinton,true
2431,Nell Benton,"""I'm for a three-state political solution in I...",Sam Brownback,true
2432,Wes Allison,"""We now have the greatest income inequality si...",John Edwards,true
2433,Lissa August,"""Tommy Thompson is the father of welfare refor...",Tommy Thompson,true


In [9]:
#Create a function to get a binary number from the target
def getBinaryNumTarget(text):
    if text == 'true' :
        return 1
    
    else:
        return 0

#Create a function to get only true or false values from the target
def getBinaryTarget(text):
    if text == 'true' or text=='mostly-true' :
        return 'REAL'
    else:
        return 'FAKE'

In [68]:
#Store the data in the dataframe
data['BinaryTarget'] = data['target'].apply(getBinaryTarget)
data['BinaryNumTarget'] = data['target'].apply(getBinaryNumTarget)


In [69]:
#Show the data
data

,author,statement,source,target,BinaryTarget,BinaryNumTarget
0,Jill Terreri,"New York is ""leading the nation with the lowes...",NYS Department of Corrections and Community Su...,true,REAL,1
1,Eleanor Hildebrandt,“Iowa ranks 45th in the U.S. for internet conn...,Cindy Axne,true,REAL,1
2,Emily Zirkelbach,“FUN FACT: West Virginia is home to zero billi...,David Gura,true,REAL,1
3,Madeline Heim,Says gubernatorial candidate Rebecca Kleefisch...,Kelda Helen Roys,true,REAL,1
4,Sabine Martin,"The Biden administration ""published a study co...",Randy Feenstra,true,REAL,1
...,...,...,...,...,...,...
2430,Bill Adair,"""Corporate profits are up, CEO pay is up, but ...",Hillary Clinton,true,REAL,1
2431,Nell Benton,"""I'm for a three-state political solution in I...",Sam Brownback,true,REAL,1
2432,Wes Allison,"""We now have the greatest income inequality si...",John Edwards,true,REAL,1
2433,Lissa August,"""Tommy Thompson is the father of welfare refor...",Tommy Thompson,true,REAL,1


In [70]:
data.to_csv('political_realfacts.csv')

# SCRAPING FAKE NEWS

In [3]:
#Create a function to scrape the site
def scrape_websites(page_number):
    page_num = str(page_number) #Convert the page number to a string
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num+'&ruling=false' #append the page number to complete the URL
    webpage = requests.get(URL)  #Make a request to the website
    time.sleep(3)
    soup = BeautifulSoup(webpage.text, "html.parser") #Parse the text from the website
    
    #Get the tags and it's class
    statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'})  #Get the tag and it's class
    statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) #Get the tag and it's class
    statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#Get the tag and it's class
    target = soup.find_all('div', attrs={'class':'m-statement__meter'}) #Get the tag and it's class
    
    #loop through the footer class m-statement__footer to get the date and author
    
    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name+' '+last_name
       # month = name_and_date[4]
      #  day = name_and_date[5]
       # year = name_and_date[6]
       # date = month+' '+day+' '+year
        #dates.append(date)
        authors.append(full_name)
        
    #Loop through the div m-statement__quote to get the link
    for i in statement_quote:
        link2 = i.find_all('a')
        statements.append(link2[0].text.strip())
    #Loop through the div m-statement__meta to get the source
    for i in statement_meta:
        link3 = i.find_all('a') #Source
        source_text = link3[0].text.strip()
        sources.append(source_text)
    #Loop through the target or the div m-statement__meter to get the facts about the statement (True or False)
    for i in target:
        fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(fact)

In [4]:
#Loop through 'n-1' webpages to scrape the data
n=150
for i in range(1, n):
    scrape_websites(i)

In [5]:
#Create a new dataFrame 
data1 = pd.DataFrame(columns = ['author',  'statement', 'source', 'target']) 
data1['author'] = authors
data1['statement'] = statements
data1['source'] = sources
#data['date'] = dates
data1['target'] = targets
#Show the data set
data1

,author,statement,source,target
0,Samantha Putterman,"Tylenol ""ruins your ability to kill” COVID-19.",Instagram posts,false
1,Ciara O'Rourke,“Chief Justice John Roberts goes after the hig...,Bloggers,false
2,Monique Curet,“Pfizer CEO: New Pill Will Have a Microchip Th...,Bloggers,false
3,Tom Kertscher,President Joe Biden “never had a” COVID-19 “pl...,Save America PAC,false
4,Ciara O'Rourke,President Joe Biden “has the lowest approval r...,Facebook posts,false
...,...,...,...,...
4465,Louis Jacobson,"The Massachusetts health care plan is ""wildly ...",Laura Ingraham,false
4466,Ciara O'Rourke,"Says judges are using Shariah law in Dearborn,...",Leo Berman,false
4467,Sean Gorman,"George Allen ""had 40,000 earmarks"" while he wa...",Jamie Radtke,false
4468,Tom Kertscher,"Says Wisconsin Gov. Scott Walker ""literally in...",Gwen Moore,false


In [10]:
data1['BinaryTarget'] = data1['target'].apply(getBinaryTarget)
data1['BinaryNumTarget'] = data1['target'].apply(getBinaryNumTarget)


In [11]:
data1

,author,statement,source,target,BinaryTarget,BinaryNumTarget
0,Samantha Putterman,"Tylenol ""ruins your ability to kill” COVID-19.",Instagram posts,false,FAKE,0
1,Ciara O'Rourke,“Chief Justice John Roberts goes after the hig...,Bloggers,false,FAKE,0
2,Monique Curet,“Pfizer CEO: New Pill Will Have a Microchip Th...,Bloggers,false,FAKE,0
3,Tom Kertscher,President Joe Biden “never had a” COVID-19 “pl...,Save America PAC,false,FAKE,0
4,Ciara O'Rourke,President Joe Biden “has the lowest approval r...,Facebook posts,false,FAKE,0
...,...,...,...,...,...,...
4465,Louis Jacobson,"The Massachusetts health care plan is ""wildly ...",Laura Ingraham,false,FAKE,0
4466,Ciara O'Rourke,"Says judges are using Shariah law in Dearborn,...",Leo Berman,false,FAKE,0
4467,Sean Gorman,"George Allen ""had 40,000 earmarks"" while he wa...",Jamie Radtke,false,FAKE,0
4468,Tom Kertscher,"Says Wisconsin Gov. Scott Walker ""literally in...",Gwen Moore,false,FAKE,0


In [8]:
data1.to_csv('political_fakefacts.csv')